# Проект по курсу "Рекомендательные системы"

## "Дружеское пари". Класс моделей: матричные факторизации
  
Правила заполнения ноутбуков на авто-проверку:
- повторить окружение преподавателя
```bash
pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0
```
- не добавлять новые импорты и не использовать дополнительные библиотеки. В противном случае ноутбук не пройдёт проверку и получит `0` баллов
- писать код только в ячейках с пометкой # YOUR CODE HERE, сразу после этой пометки
- не менять код преподавателя
- не добавлять новые ячейки
- следить, чтобы не было warning - они автоматом фейлят задание
- перед сдачей проверить, что весь ноутбук прогонятся от начала до конца и все тесты проходят

**В данном проекте вам нужно будет подбирать гипер-параметры моделей. Писать код для подбора гипер-параметров, использовать optuna и т.п. рекомендуем в отдельном ноутбуке.**

**Библиотеки implicit и lightfm не фиксируют random state при num_threads > 1. Если результат работы модели не сильно превышает необходимомый порог и рандом может опустить его ниже требуемого уровня, рекомендуем продолжить повышение качества модели: тюнинг гипер-параметров, подбор фичей, подбор метода обработки датасета**

## Импорты и данные

In [91]:
!python -V

Python 3.11.8


In [92]:
pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0

Note: you may need to restart the kernel to use updated packages.


In [93]:
# Убедитесь, что вы не добавляете новые импорты в ноутбук. Решение должно быть ограничено данными библиотеками

import warnings
warnings.simplefilter("ignore")

import implicit
import rectools
import pandas as pd
import numpy as np
import scipy
import requests

print(implicit.__version__)
print(rectools.__version__)
print(pd.__version__)
print(np.__version__)
print(scipy.__version__)
print(requests.__version__)

0.7.2
0.12.0
2.2.3
1.26.4
1.12.0
2.32.3


In [94]:
import os.path

from implicit.als import AlternatingLeastSquares
from lightfm import LightFM

from rectools import Columns
from rectools.metrics import MAP, MeanInvUserFreq
from rectools.dataset import Dataset
from rectools.models import PureSVDModel, ImplicitALSWrapperModel, LightFMWrapperModel, model_from_config

# For implicit ALS
import os
import threadpoolctl
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

Если у вас нет данных, то используйте закомментированный код

In [95]:
from tqdm.auto import tqdm
import zipfile as zf

url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
     for chunk in req.iter_content(chunk_size=2 ** 20):
         progress_bar.update(len(chunk))
         fd.write(chunk)

files = zf.ZipFile('kion.zip', 'r')
files.extractall()
files.close()

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [96]:
data_path = os.environ.get("DATA_PATH")
if data_path is None:
    data_path = "data_original"  # ваш путь к данным до папки data_original включительно (поменяйте при необходимости)

In [97]:
interactions = (
    pd.read_csv(os.path.join(data_path, "interactions.csv"), parse_dates=["last_watch_dt"])
    .rename(columns={'total_dur': Columns.Weight,
                     'last_watch_dt': Columns.Datetime})
)
users = pd.read_csv(os.path.join(data_path, "users.csv"))
items = pd.read_csv(os.path.join(data_path, "items.csv"))

print(interactions.shape)
interactions.head(5)

(5476251, 5)


user_id  item_id   datetime  weight  watched_pct
0   176549     9506 2021-05-11    4250         72.0
1   699317     1659 2021-05-29    8317        100.0
2   656683     7107 2021-05-09      10          0.0
3   864613     7638 2021-07-05   14483        100.0
4   964868     9506 2021-04-30    6725        100.0

In [98]:
N_DAYS = 7

max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] <= max_date - pd.Timedelta(days=N_DAYS))]
test = interactions[(interactions['datetime'] > max_date - pd.Timedelta(days=N_DAYS))]

catalog = train[Columns.Item].unique()

test_users = test[Columns.User].unique()
cold_users = set(test_users) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[Columns.User].unique()
print(test.shape[0])
print(test[Columns.User].nunique())

306752
111240


## ImplicitALS (16 баллов)

### Ситуация:

Коллега вернулся из отпуска и вы вместе сели за улучшение модели. Внимательно изучив репозиторий библиотеки implicit вы увидели модель iALS и решаете попробовать ее в деле.

Чтобы работа была интереснее, вы заключаете пари с вашим коллегой о том, кто выбьет больше MAP@K на горячих пользователях. 

Правила пари: 
- Валидируемся на последней неделе (переменная `test`) и на горячих пользователях `hot_users`
- Можно собрать свой `Dataset` на основе `train`, трансформированного, если нужно
- Параметры модели задаются конфигом, которые будут передаваться в `model_from_config`

У вашего коллеги получилось выбить на ImplicitALS `MAP@K = 0.052`. Ваша задача побить его рекорд.

In [99]:
def get_dataset(train: pd.DataFrame) -> Dataset:
    df = (
        train
        .groupby([Columns.User, Columns.Item])
        .agg({
            Columns.Weight: "sum",
            Columns.Datetime: "max"
        })
        .reset_index()
    )

    df[Columns.Weight] = np.log1p(df[Columns.Weight])

    dataset = Dataset.construct(
        interactions_df=df[[Columns.User, Columns.Item, Columns.Weight, Columns.Datetime]]
    )
    # YOUR CODE HERE
    return dataset

config = {
    "cls": "ImplicitALSWrapperModel",
    "model": {
        "factors": 10,               #<
        "regularization": 0.01,       
        "iterations": 50,             
        "alpha": 10,                  
        "use_gpu": False,            
        "random_state": 123,           
        "dtype": "float32"           
    }
}

In [100]:
%%time
model = model_from_config(config)
dataset = get_dataset(train.copy())

assert config['cls'] == 'ImplicitALSWrapperModel'
assert dataset.item_features is None
assert dataset.user_features is None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.052

0.057148252838149
CPU times: user 1min 4s, sys: 1.06 s, total: 1min 5s
Wall time: 1min 4s


## SVD (16 баллов)

На ваш громкий спор с коллегой о том, что все дело в вашем удачном random seed, к вам подошел ваш лид. 

Узнав детали вашего спора, он дает вам комментарий, что iALS хороша, но погружение в матричную факторизацию следует начинать с `SVD`.

Вы переглянулись с коллегой и решаете уладить спор о random seed во втором раунде, используя новую модель.

Ваш коллега смогу выбить на SVD `MAP@K = 0.066`. Вы знаете, что делать.

In [101]:
def get_dataset(train: pd.DataFrame) -> Dataset:
    df = (
        train
        .groupby([Columns.User, Columns.Item])
        .agg({Columns.Weight: "sum", Columns.Datetime: "max"})
        .reset_index()
    )
    df[Columns.Weight] = np.log1p(df[Columns.Weight])

    dataset = Dataset.construct(
        interactions_df=df[[Columns.User, Columns.Item, Columns.Weight, Columns.Datetime]]
    )

    return dataset

config = {
    'cls': 'PureSVDModel',
 'verbose': 0,
 'factors': 2, #<
 'tol': 0.0,
 'maxiter': 40,
 'random_state': 0
}

In [102]:
%%time

model = model_from_config(config)
dataset = get_dataset(train.copy())

assert config['cls'] == 'PureSVDModel'
assert dataset.item_features is None
assert dataset.user_features is None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.066

0.07175820748985508
CPU times: user 7.23 s, sys: 827 ms, total: 8.06 s
Wall time: 7.82 s


## Dataset with features (8 баллов)

"Ну это ни в какие ворота!" - восклицает ваш коллега, увидев ваш победный конфиг. Из другого угла опенспейса доносится "А я говорил" от вашего лида.

В это время к вам сзади подходит продакт и интересуется предметом вашего спора.

Рассказав про особенности найденных вами моделей, он просит вас в них докинуть фичи, ведь на одних взаимодействиях далеко не уедешь.

Вы согласились, ведь это отличная возможность продолжить пари. Соберите `Dataset` с фичами по пользователям и итемам, который вы будете использовать дальше


In [103]:
def get_dataset_with_features(train: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame) -> Dataset:
    # YOUR CODE HERE

    train = train.copy()

    max_date = train["datetime"].max()
    train["days_since"] = (max_date - train["datetime"]).dt.days
    train["weight"] = 1 / (1 + train["days_since"].clip(lower=0))  # Защита от деления на 0

    # Подготовка фичей пользователей
    FEATURES_COLUMNS = ["age", "income", "sex", "kids_flg"]
    user_features_dfs = []
    
    for feature in FEATURES_COLUMNS:
        feature_df = users[[Columns.User, feature]].copy()
        feature_df.columns = ["id", "value"]
        feature_df["feature"] = feature
        user_features_dfs.append(feature_df)

    user_features_df = pd.concat(user_features_dfs, ignore_index=True)

    # Подготовка фичей товаров
    items = items.copy()
    # Жанры
    items["genre"] = items["genres"].str.lower().str.split(", ")
    genre_feature = items[[Columns.Item, "genre"]].explode("genre")
    genre_feature.columns = ["id", "value"]
    genre_feature["feature"] = "genre"
    
    # Страны
    items["countries"] = items["countries"].str.lower().str.split(", ")
    countries_feature = items[[Columns.Item, "countries"]].explode("countries")
    countries_feature.columns = ["id", "value"]
    countries_feature["feature"] = "countries"
    
    # Тип контента
    content_feature = items[[Columns.Item, "content_type"]].copy()
    content_feature.columns = ["id", "value"]
    content_feature["feature"] = "content_type"

    items_features_df = pd.concat([content_feature, genre_feature, countries_feature], ignore_index=True)

    # Создание датасета
    dataset = Dataset.construct(
        interactions_df=train,
        user_features_df=user_features_df,
        item_features_df=items_features_df,
        cat_user_features=["sex", "age", "income", "kids_flg"],
        cat_item_features=["genre", "content_type", "countries"]
    )
    
    return dataset

In [104]:
dataset_with_features = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert (dataset_with_features.user_features is not None) and (dataset_with_features.item_features is not None)

## ImplicitALS with features (20 баллов)

Собрав датасет с фичами, вы готовы к третьему раунду пари. 

Вы решаете начать снова с `iALS`, до сих пор удивляясь результатам модели `SVD`.

Ваш коллега изучил вашу технику подбора random seed и хитро улыбается вам.

Он смог выбить `MAP@K = 0.073`, теперь ваш ход.

In [105]:
config = {
    'cls': 'ImplicitALSWrapperModel',
    'model': {
        'cls': 'AlternatingLeastSquares',
        'factors': 30,
        'random_state': 42,
        'num_threads': 16,
    },
    'fit_features_together': True  # или False
}

In [106]:
%%time
model = model_from_config(config)
dataset = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert config['cls'] == 'ImplicitALSWrapperModel'
assert dataset.item_features is not None
assert dataset.user_features is not None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.073

0.08116735898314537
CPU times: user 8min 34s, sys: 4.8 s, total: 8min 39s
Wall time: 8min 20s


## LightFM with features (20 баллов)

И снова ор выше гор, ваш пайплайн подготовки датасета помог вам в очередной раз обойти вашего коллегу.

Не зная, к чему еще аппелировать, он зовет вашего лида, чтобы тот внимательно изучил полученные результаты.

"iALS с фичами это хорошо, но тут стоит попробовать факторизационные машины, попробуйте `LightFM`" - заключает он. Вы переключаетесь на изучение новой библиотеки, предвкушая финальный раунд.

Ваш коллега смог выжать из своего обновленного `Dataset` и `LightFM` скор `MAP@10 = 0.08`. Последний рывок.

In [107]:
def get_dataset_with_features(train: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame) -> Dataset:
    # YOUR CODE HERE

    train = train.copy()

    max_date = train["datetime"].max()
    train["days_since"] = (max_date - train["datetime"]).dt.days
    train["weight"] = 1 / (1 + train["days_since"].clip(lower=0))  # Защита от деления на 0

    # Подготовка фичей пользователей
    FEATURES_COLUMNS = ["age", "income", "sex", "kids_flg"]
    user_features_dfs = []
    
    for feature in FEATURES_COLUMNS:
        feature_df = users[[Columns.User, feature]].copy()
        feature_df.columns = ["id", "value"]
        feature_df["feature"] = feature
        user_features_dfs.append(feature_df)

    user_features_df = pd.concat(user_features_dfs, ignore_index=True)

    # Подготовка фичей товаров
    items = items.copy()
    # Жанры
    items["genre"] = items["genres"].str.lower().str.split(", ")
    genre_feature = items[[Columns.Item, "genre"]].explode("genre")
    genre_feature.columns = ["id", "value"]
    genre_feature["feature"] = "genre"
    
    # Страны
    items["countries"] = items["countries"].str.lower().str.split(", ")
    countries_feature = items[[Columns.Item, "countries"]].explode("countries")
    countries_feature.columns = ["id", "value"]
    countries_feature["feature"] = "countries"
    
    # Тип контента
    content_feature = items[[Columns.Item, "content_type"]].copy()
    content_feature.columns = ["id", "value"]
    content_feature["feature"] = "content_type"

    items_features_df = pd.concat([content_feature, genre_feature, countries_feature], ignore_index=True)

    # Создание датасета
    dataset = Dataset.construct(
        interactions_df=train,
        user_features_df=user_features_df,
        item_features_df=items_features_df,
        cat_user_features=["sex", "age", "income", "kids_flg"],
        cat_item_features=["genre", "content_type", "countries"]
    )
    
    return dataset

config = {
    "cls": "LightFMWrapperModel",
    "model": {
        "loss": "warp",
        "no_components": 100,
        "item_alpha": 0.005,
        "user_alpha": 0.005,
        "random_state": 42
    },
    "epochs": 30,
    "num_threads": 4,
    "recommend_n_threads": 4
}

In [ ]:
%%time
model = model_from_config(config)
dataset = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert config['cls'] == 'LightFMWrapperModel'
assert dataset.item_features is not None
assert dataset.user_features is not None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.08